<a href="https://colab.research.google.com/github/tienhuynh96/End-to-end-Question-Answering/blob/main/%5BDemo%5D_HF_Faiss_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **1. Install and import bibraries**

In [1]:
!pip install -qq transformers[sentencepiece]==4.35.2 datasets==2.16.1 evaluate==0.4.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.2 MB/s eta 0:00:00


In [2]:
# Instal faiss-gpu: Facebook AI Similarity Search
!sudo apt-get install libomp-dev
!pip install -qq faiss-gpu

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libomp-14-dev libomp5-14
Suggested packages:
  libomp-14-doc
The following NEW packages will be installed:
  libomp-14-dev libomp-dev libomp5-14
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 738 kB of archives.
After this operation, 8,991 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libomp5-14 amd64 1:14.0.0-1ubuntu1.1 [389 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libomp-14-dev amd64 1:14.0.0-1ubuntu1.1 [347 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libomp-dev amd64 1:14.0-55~exp2 [3,074 B]
Fetched 738 kB in 0s (2,370 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [3]:
import numpy as np
import collections
import torch
import faiss      # Cài thêm thư viện faiss
import evaluate

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel
from transformers import AutoModelForQuestionAnswering
from transformers import TrainingArguments
from transformers import Trainer
from tqdm.auto import tqdm

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


## **2. Download dataset**

In [5]:
DATASET_NAME = 'squad_v2'
# Chia nhỏ 40 phần và chỉ lấy phần đầu: shard(num_shards = 40, index=0)
raw_datasets = load_dataset(DATASET_NAME, split='train').shard(num_shards = 40, index=0)
raw_datasets

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 3258
})

## **3. Filter out non-answerable samples**

In [6]:
raw_datasets = raw_datasets.filter(
    lambda x: len(x['answers']['text']) > 0
)

raw_datasets

Filter:   0%|          | 0/3258 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 2175
})

## **4. Intialize pre-trained model**

In [7]:
MODEL_NAME = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

## **5. Create get vector embedding functions**

In [15]:
# Lấy giá trị của token cls và xem như giá trị vector embedding của text
def cls_pooling(model_output):
  return model_output.last_hidden_state[:, 0]      # last_hiden_state = batch size, sequence length => vị trí "0" là cls

In [16]:
encoded_input = tokenizer(
    raw_datasets['question'][0],
    padding=True,
    truncation=True,
    return_tensors='pt'
)

encoded_input

{'input_ids': tensor([[  101,  2043,  2106, 20773,  2707,  3352,  2759,  1029,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [19]:
def get_embeddings(text_list):
  encoded_input = tokenizer(
      text_list,
      padding=True,
      truncation=True,
      return_tensors='pt'
  )

  encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
  #  '**' is the "dictionary unpacking" operator,
  # Pair  input_ids, attention_mask and its value in dictionary is equivalent to the required input and its value in model ( input_ids = ..., attention_mask=...)
  model_output = model(**encoded_input)

  return cls_pooling(model_output)

In [21]:
# Test functionality
embedding = get_embeddings(raw_datasets['question'][0])
embedding.shape

torch.Size([1, 768])

In [27]:
# Convert to numpy array (required for HF Datasets)
EMBEDDING_COLUMN = 'question_embedding'
embeddings_dataset = raw_datasets.map(
    lambda x: {EMBEDDING_COLUMN: get_embeddings(x['question']).detach().cpu().numpy()[0]}
)
# detach() mean: create a tensor that shares storage with the original tensor but without tracking operations.

In [28]:
embeddings_dataset

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'question_embedding'],
    num_rows: 2175
})

In [29]:
# Enhance the dataset with a FAISS index, which will facilitate fast and efficient similarity searches.
embeddings_dataset.add_faiss_index(column=EMBEDDING_COLUMN)

  0%|          | 0/3 [00:00<?, ?it/s]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'question_embedding'],
    num_rows: 2175
})

## **6. Search similar samples with a question**

In [26]:
# Search question
question = 'When did Beyonce start becoming popular?'

input_quest_embedding = get_embeddings([question]).cpu().detach().numpy()
input_quest_embedding.shape

(1, 768)

In [31]:
# Search top 5 similar examples
TOP_K = 5

scores, samples = embeddings_dataset.get_nearest_examples(
    EMBEDDING_COLUMN, input_quest_embedding, k = TOP_K
)

In [33]:
for idx, score in enumerate(scores):
  print(f'Top {idx + 1}\tScore: {score}')
  print(f'Question: {samples["question"][idx]}')
  print(f'Context: {samples["context"][idx]}')
  print(f'Answer: {samples["answers"][idx]}')
  print()

Top 1	Score: 0.0
Question: When did Beyonce start becoming popular?
Context: Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".
Answer: {'text': ['in the late 1990s'], 'answer_start': [269]}

Top 2	Score: 7.510955333709717
Question: What movie did Beyonce act in 2006?
Context: Her first acting role of 2006 was in the comedy film The Pink Panther starring